In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

print("Using TensorFlow version:", tf.__version__)

# Pour reproduire un comportement stable (optionnel)
tf.random.set_seed(42)

## 1. Chargement du dataset

In [ ]:
# Chemin vers le dossier qui contient les deux sous-dossiers : 'good' et 'defect'
DATA_DIR = "path_to_your_mvtec_dataset"  # À adapter

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Charger toutes les images d'un seul tenant
dataset = image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='categorical',  # Pour un problème multi-classes, ici on suppose 2 classes (good, defect)
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True  # Mélange les données
)

# Afficher les classes détectées
class_names = dataset.class_names
print("Classes détectées :", class_names)

# Optionnel : Normalisation et mise en cache
AUTOTUNE = tf.data.AUTOTUNE
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalisation simple
    return image, label

dataset = dataset.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

## 2. Définition d'un modèle simple
On définit un modèle CNN

In [ ]:
num_classes = len(class_names)

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

model.summary()

## 3. Entraînement du modèle

In [ ]:
EPOCHS = 1000  # Nombre d'époques d'entraînement

history = model.fit(
    dataset,                 # Entraînement ET test sur le même dataset
    epochs=EPOCHS,
    verbose=1
)

## 4. Évaluation sur le même dataset

In [ ]:
loss, accuracy = model.evaluate(dataset, verbose=0)
print(f"Loss sur le dataset complet : {loss:.4f}")
print(f"Accuracy sur le dataset complet : {accuracy:.4f}")